In [38]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset, load_from_disk
from transformers import DataCollatorForLanguageModeling
import transformers

tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path="openai-community/gpt2",
    use_fast=True,
    trust_remote_code=True, 
    cache_dir="./cache"
)

def tokenize(examples):
    return tokenizer(examples["prompt"])

len(tokenizer)
#dataset = load_dataset("fka/awesome-chatgpt-prompts", cache_dir="./cache", trust_remote_code=True)

c:\Users\mingx\anaconda3\envs\mingxuan\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


50257

In [18]:
datasets = load_from_disk("./preprocessed_data")
datasets["train"][0]

{'input_ids': [4561, 4233, 3594, 32019, 290, 12205, 332, 33074],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [4561, 4233, 3594, 32019, 290, 12205, 332, 33074]}

In [39]:
model.get_input_embeddings().weight.shape[0]

50257

In [5]:
transformers.__version__


'4.44.2'

In [36]:
import torch
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="openai-community/gpt2",
    trust_remote_code=True,
    cache_dir="./cache",
    attn_implementation="sdpa",
    torch_dtype=torch.float16,
    device_map="auto"
)
model.device

device(type='cuda', index=0)

In [37]:
optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr=1e-4, 
    betas=[0.9, 0.95], 
    eps=1e-8,
    weight_decay=0.0,
    fused=True
)

In [75]:
model.dtype

torch.float32

In [64]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [66]:
tokenized_subset = dataset["train"].map(
    tokenize, batched=True, remove_columns=dataset["train"].column_names
)

Map: 100%|██████████| 170/170 [00:00<00:00, 5364.04 examples/s]


In [47]:
out = data_collator([tokenized_subset[i]for i in range(5)])

In [50]:
for key in out:
    print(f"{key} shape: {out[key].shape}")

input_ids shape: torch.Size([5, 708])
attention_mask shape: torch.Size([5, 708])
labels shape: torch.Size([5, 708])
